In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt

import pickle
import tensorflow as tf
import numpy as np

from utils.Param import FTFYParam
from network.dataset.ftfy_patchdata import input_fn
from network.model_fn import ftfy_model_fn
from network.train import FTFYEstimator

from utils.eval import calc_iou_k, ftfy_retrieval_accuracy

from tqdm import tqdm

In [ ]:
# set seed for reproduction
np.random.seed(2019)
tf.set_random_seed(2019)

In [ ]:
# parameters (adjust as needed)
param = FTFYParam(ftfy_scope='ftfy', feat_scope='triplet-net', log_dir=None)
param.is_ftfy_model = True
param.batch_size = 8 # 32 for v100
feat_trainable = False

log_dir = './log/sem_ftfy_full'
param.model_path = './log/sem_ftfy_full/ckpt' # './log/sem'
param.train_datasets = 'sem' 
param.test_datasets = 'human'

is_sem = param.test_datasets == 'sem'
if is_sem:
    param.data_dir = '/home/sungsooha/Desktop/Data/ftfy/sem/train'
else:
    param.data_dir = '/home/sungsooha/Desktop/Data/ftfy/austin'
    #param.train_datasets = 'campus'
    param.src_dir = 'human_sources'
    param.tar_dir = 'human_patch'
    param.tar_patches_per_row = 13
    param.tar_patches_per_col = 6

In [ ]:
if is_sem:
    # only for sem dataset
    data_dirs = []
    for data_dir in os.listdir(param.data_dir):
        if os.path.isdir(os.path.join(param.data_dir, data_dir)):
            data_dirs.append(data_dir)
    data_dirs = sorted(data_dirs)
else:
    data_dirs = ['.']

### data pipeline

In [ ]:
tf.logging.info("Preparing data pipeline ...")
with tf.device('/cpu:0'), tf.name_scope('input'):
    dataset, data_sampler = input_fn(
        param.data_dir,
        batch_size=param.batch_size,
        cellsz=param.src_cellsz,
        n_parameters=param.n_parameters,
        src_size=param.src_size,
        tar_size=param.tar_size,
        n_channels=param.n_channels
    )
    data_iterator = tf.data.Iterator.from_structure(
        dataset.output_types,
        dataset.output_shapes
    )
    dataset_init = data_iterator.make_initializer(dataset)
    batch_data = data_iterator.get_next()

data_sampler.load_dataset(
    data_dirs, param.src_dir, param.tar_dir,
    src_ext=param.src_ext, src_size=param.src_size, n_src_channels=param.n_channels,
    src_per_col=param.src_patches_per_col, src_per_row=param.src_patches_per_row,
    tar_ext=param.tar_ext, tar_size=param.tar_size, n_tar_channels=param.n_channels,
    tar_per_col=param.tar_patches_per_col, tar_per_row=param.tar_patches_per_row
)

### compute data statistics

In [ ]:
tf.logging.info('Loading training stats: %s' % param.train_datasets)
file = open(os.path.join(log_dir, 'stats_%s.pkl' % param.train_datasets), 'rb')
mean, std = pickle.load(file)
tf.logging.info('Mean: {:.5f}'.format(mean))
tf.logging.info('Std : {:.5f}'.format(std))
data_sampler.normalize_data(mean, std)

### build model

In [ ]:
tf.logging.info("Creating the model ...")
sources, targets, labels, bboxes = batch_data
spec = ftfy_model_fn(sources, targets, labels, bboxes,
                     mode='TEST', **param.get_model_kwargs(feat_trainable=feat_trainable))
# 20-th epoch, logged with 5 interval
estimator = FTFYEstimator(spec, **param.get_ckpt_kwargs())

### Testing

In [ ]:
top_k = [1, 5, 10, 20]
iou_thrs = [0.7, 0.8, 0.9]
n_iters = 10
n_max_tests = 5000

avg_accuracy = np.zeros((len(iou_thrs), len(top_k)), dtype=np.float32)
avg_d_mean = np.zeros(4, dtype=np.float32)
avg_d_std = np.zeros(4, dtype=np.float32)

for it in range(n_iters):
    tf.logging.info('Try {:d}'.format(it))
    
    data_sampler.reset()
    pred_confidences, pred_bboxes, bboxes = \
        estimator.run(dataset_init,top_k=top_k[-1], n_max_test=n_max_tests)
    iou_k = calc_iou_k(pred_bboxes, bboxes)
    accuracy = ftfy_retrieval_accuracy(iou_k, top_k, iou_thrs)
    
    pred_bboxes = pred_bboxes[:, 0]
    d_bboxes = np.abs(pred_bboxes - bboxes)
    src_h, src_w = param.src_size
    d_bboxes[..., 0] *= src_w
    d_bboxes[..., 1] *= src_h
    d_bboxes[..., 2] *= src_w
    d_bboxes[..., 3] *= src_h
    d_cx_mean, d_cx_std = np.mean(d_bboxes[..., 0]), np.std(d_bboxes[..., 0])
    d_cy_mean, d_cy_std = np.mean(d_bboxes[..., 1]), np.std(d_bboxes[..., 1])
    d_w_mean, d_w_std = np.mean(d_bboxes[..., 2]), np.std(d_bboxes[..., 2])
    d_h_mean, d_h_std = np.mean(d_bboxes[..., 3]), np.std(d_bboxes[..., 3])
    
    tf.logging.info('Accuracy:')
    tf.logging.info('{}'.format(accuracy))

    tf.logging.info('For the best (@k=1), [mean, std]')
    tf.logging.info('d_cx: {:.3f}, {:.3f}'.format(d_cx_mean, d_cx_std))
    tf.logging.info('d_cy: {:.3f}, {:.3f}'.format(d_cy_mean, d_cy_std))
    tf.logging.info('d_w : {:.3f}, {:.3f}'.format(d_w_mean, d_w_std))
    tf.logging.info('d_h : {:.3f}, {:.3f}'.format(d_h_mean, d_h_std))
        
    avg_accuracy += accuracy
    avg_d_mean += np.array([d_cx_mean, d_cy_mean, d_w_mean, d_h_mean])
    avg_d_std += np.array([d_cx_std, d_cy_std, d_w_std, d_h_std])

avg_accuracy /= n_iters
avg_d_mean /= n_iters
avg_d_std /= n_iters

### Plot results

In [ ]:
avg_accuracy

In [ ]:
all_accuracy = np.asarray(avg_accuracy)
all_accuracy = np.squeeze(avg_accuracy)
plt.plot(avg_accuracy.T)

In [ ]:
tf.logging.info('For the best (@k=1), [cx, cy, w, h]')
tf.logging.info('mean: {}'.format(avg_d_mean))
tf.logging.info('std : {}'.format(avg_d_std))

In [ ]:
# save results
out_dir = os.path.join(log_dir, 'eval_metrics_{}_{}.npy'.format(
    param.train_datasets, param.test_datasets
))
metric = dict(
    accuracy=avg_accuracy,
    d_mean=avg_d_mean,
    d_std=avg_d_std
)
np.save(out_dir, metric)